In [1]:
import numpy as np
import toytree
from scipy.optimize import minimize
from scipy.linalg import expm

In [3]:
class Pagel:
    def __init__(self, tree, data):
        self.tree = tree
        self.data = data

    def assign_tip_like_values(self):
        """
        Assigns likelihood values to tree tips based on data. 
        """
        values = [{0:-(i-1),1:i} for i in self.data]
        keys = list(range(0, len(self.data), 1))
        valuesdict = dict(zip(keys,values))
        self.tree = self.tree.set_node_values(feature = "likelihood", values = valuesdict)
        return self.tree


    def cond_like(self, likeleft0, likeleft1, likeright0, likeright1, tL, tR, alpha, beta):
        """
        Calculates conditional likelihood of character states at each node
        """
        Q = np.array([[-alpha, alpha], [beta, -beta]])
        probleft = expm(Q*tL)
        probright = expm(Q*tR)

        #ancestor is 0
        left0 = probleft[0, 0] * likeleft0 + probleft[0, 1] * likeleft1
        right0 = probright[0, 0] * likeright0 + probright[0, 1] * likeright1
        like_zero = left0*right0

        #ancestor is 1
        left1 = probleft[1, 0] * likeleft0 + probleft[1, 1] * likeleft1
        right1 = probright[1, 0] * likeright0 + probright[1, 1] * likeright1
        like_one = left1*right1

        return {0: like_zero, 1: like_one}

    def pruning_alg_before(self, alpha=0.5, beta=0.5):
        for node in self.tree.treenode.traverse("postorder"):
            if len(node.children) == 2:
                child1 = node.children[0]
                child2 = node.children[1]
                likedict = self.cond_like(likeright0 = child1.likelihood[0],
                                     likeright1 = child1.likelihood[1],
                                     likeleft0 = child2.likelihood[0],
                                     likeleft1 = child2.likelihood[1],
                                     tR = child1.dist,
                                     tL = child2.dist,
                                     alpha = alpha,
                                     beta = beta)
                #print(likedict)
                node.likelihood = likedict


    def node_like(self, x0, likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
        """
        Calculates total likelihood (likelihood of the node having the character state
        0 or the character state 1) at each node.
        """

        condlik = self.cond_like(likeleft0, likeleft1, likeright0, likeright1, tL, tR, x0[0], x0[1])

        # get full likelihood
        lik = (1 - anca) * condlik[0] + (anca) * condlik[1]
        if anca in [0., 1.]:
            lik /= 2

        return -lik #np.log(lik)

    def model_fit(self, likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
        """
        Find the maximum likelihood estimate of the two
        rate model parameters given the data.
        """
        args = (likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca)

        # ML estimate
        estimate = minimize(
            fun=self.node_like, 
            x0=np.array([1., 1.]),
            args=args,
            method='L-BFGS-B',
            bounds=((0, 10), (0, 10))
            )

        score = -1 * self.node_like(estimate.x, *args)
        result = {
            "alpha": round(estimate.x[0], 3),
            "beta": round(estimate.x[1], 3), 
            "lik": round(score, 3),
            "convergence": estimate.success,
            }
        return result

    def fit_model_at_nodes(self, anca = 0.5):
        """
        Fits the Markov state model at each individual node, traversing the tree in postorder.
        Stores alpha and beta values (instantaneous transition rates) at each node.
        """
        self.tree = self.tree.set_node_values('alpha')
        self.tree = self.tree.set_node_values('beta')
        for node in self.tree.treenode.traverse("postorder"):
            if len(node.children) == 2:
                child1 = node.children[0]
                child2 = node.children[1]
                model = self.model_fit(likeright0 = child1.likelihood[0],
                                  likeright1 = child1.likelihood[1],
                                  likeleft0 = child2.likelihood[0],
                                  likeleft1 = child2.likelihood[1],
                                  tR = child1.dist,
                                  tL = child2.dist,
                                  anca = anca)
                node.alpha = model['alpha']
                node.beta = model['beta']
        return self.tree

    def pruning_alg_after(self):
        """
        Runs Felsenstein's pruning algorithm on an input tree, given instantaneous transition
        rates alpha and beta. Assigns likelihood scores for characters states at each node.
        Specifically for binary state model. 
        """
        self.tree = self.fit_model_at_nodes()
        for node in self.tree.treenode.traverse("postorder"):
            if len(node.children) == 2:
                child1 = node.children[0]
                child2 = node.children[1]
                likedict = self.cond_like(likeright0 = child1.likelihood[0],
                                     likeright1 = child1.likelihood[1],
                                     likeleft0 = child2.likelihood[0],
                                     likeleft1 = child2.likelihood[1],
                                     tR = child1.dist,
                                     tL = child2.dist,
                                     alpha = float(node.alpha),
                                     beta = float(node.beta))
                node.likelihood = likedict
        return self.tree
   
    def run(self):
        """
        Assigns likelihood values to tips and runs Felstein's pruning algorithm for 
        a list of data along the provided tree
        """
        self.assign_tip_like_values()
        #I don't understand why it needs like values at nodes to run the pruning alg
        #after model_fit_at_nodes
        self.pruning_alg_before()
        self.tree = self.pruning_alg_after()

In [17]:
testtree = toytree.rtree.unittree(ntips=10)
testtree.newick

'((r9:0.8,(r8:0.6,(r7:0.4,r6:0.4)100:0.2)100:0.2)100:0.2,((r5:0.6,r4:0.6)100:0.2,(r3:0.6,(r2:0.4,(r1:0.2,r0:0.2)100:0.2)100:0.2)100:0.2)100:0.2);'

In [18]:
testdata = [1,0,1,1,1,0,0,1,0,1]

In [19]:
test = Pagel(tree=testtree,data=testdata)
test.run()
test.tree.get_node_values('likelihood',True,True)

array([{0: 0.013909902448584229, 1: 0.015250741153785555},
       {0: 0.2175060189744712, 1: 0.24937595247663535},
       {0: 0.06243669691293588, 1: 0.06247857311294205},
       {0: 0.24999558915331158, 1: 0.24937595247663535},
       {0: 0.24999787050421876, 1: 0.24999815749065085},
       {0: 0.24867870955685062, 1: 0.24991613434302432},
       {0: 0.24999558915331163, 1: 0.24999588866552108},
       {0: 0.24533876067392907, 1: 0.24991613434302432},
       {0: 0.24991613434302426, 1: 0.24991613434302432}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0},
       {0: 0, 1: 1}], dtype=object)